In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/ML4FINANCE

/content/drive/MyDrive/ML4FINANCE


In [3]:
import torch
from transformers import pipeline
from sentiment_analysis import *

# 10-K Reports

In [ ]:
tenkreports = pd.read_parquet('datasets/Predictors/mda_text.parquet')

In [ ]:
tenkreports

In [ ]:
tenkreports['report_year'].value_counts()

We only have data for 2015.

# Chen-Zimmerman data

In [ ]:
chenz = pd.read_csv('datasets/Predictors/chenz.csv')
chenz


# Earning Calls

In [ ]:
earnings_calls = pd.read_parquet('datasets/Predictors/earnings_calls.parquet')

In [ ]:
earnings_calls

In [ ]:
earnings_calls.dtypes

# Quarterly Compustat Firm Characteristics

In [ ]:
firm_char = pd.read_csv('datasets/Predictors/CompFirmCharac.csv')

In [ ]:
firm_char

# JKP

In [ ]:
jkp = pd.read_csv('datasets/Predictors/jkp.csv')

In [ ]:
jkp

idée: C'est que le sentiment analysis va retourner uniquement un sentiment: positif, négatif ou neutre donc il n'y aura pas vraiment de data leakage. Dans les faits, dans certains datasets ils parlent des performances mais on est pas dans le data leakage car notre sentiment analysis ne resort pas de chiffre non?

# Earning Calls

In [ ]:
earnings_calls = pd.read_parquet('datasets/Predictors/earnings_calls.parquet')

In [ ]:
earnings_calls

,transcriptid,componentorder,componenttext,mostimportantdateutc,gvkey,companyname,transcriptcomponenttypename
0,3285682,0,"Good morning, everyone, and welcome to the LXI...",2020-11-23,323562,LXI REIT plc,Presentation Operator Message
1,50630,5,"Thanks, Bob. In summary 2009 was a pivotal yea...",2010-02-18,063083,Endologix LLC,Presenter Speech
2,840499,103,"Just one thing left to ask, actually. If we ta...",2015-07-28,002410,BP p.l.c.,Question
3,47300,58,"I said, I think it certainly has been a rare o...",2010-01-27,009317,SEI Investments Company,Answer
4,49778,1,<strong>Operator</strong>\nLadies and gentleme...,2010-02-12,176660,3SBio Inc.,Presentation Section
...,...,...,...,...,...,...,...
12475891,1220890,45,So a key driver of the improvement last year a...,2017-05-11,220426,Zurich Insurance Group AG,Answer
12475892,1220890,43,"On the rate increase, I mean if you look at th...",2017-05-11,220426,Zurich Insurance Group AG,Answer
12475893,1220890,40,"So I'm going to apologize in advance, Ralph. I...",2017-05-11,220426,Zurich Insurance Group AG,Answer
12475894,1220890,37,"Yes, thanks. So there -- I mean, given what I ...",2017-05-11,220426,Zurich Insurance Group AG,Answer


In [ ]:
earnings_calls.dtypes

transcriptid                            int64
componentorder                          int64
componenttext                          object
mostimportantdateutc           datetime64[us]
gvkey                                  object
companyname                            object
transcriptcomponenttypename            object
dtype: object

In [ ]:
earnings_calls.sort_values(by=['mostimportantdateutc','transcriptid', 'componentorder'], inplace = True)

In [ ]:
#earnings_calls.to_parquet('datasets/Predictors/earnings_calls_sorted.parquet')

In [ ]:
earnings_calls = pd.read_parquet('datasets/Predictors/earnings_calls_sorted.parquet')

# Sentiment Analysis

In [4]:
# Create a test dataframe to check our sentiment analysis
df = pd.DataFrame({
    'time': ['2021-07-15', '2020-07-12','2000-08-15'],
    'text': ["Stock markets rallied after the company reported better-than-expected quarterly earnings.",
             "Apple stock is expected to be stable",
             "Amazon stock will highly fall"]
})
df['time'] = pd.to_datetime(df['time'])
df

,time,text
0,2021-07-15,Stock markets rallied after the company report...
1,2020-07-12,Apple stock is expected to be stable
2,2000-08-15,Amazon stock will highly fall


In [5]:
# Convert to parquet as my class takes a parquet as input
df.to_parquet('datasets/Predictors/df.parquet')

# Rest

In [6]:
# Create an instance of my class with the created parquet file
reports = ReportsProcessing('datasets/Predictors/df.parquet', 'clean')
reports.data

,time,text
0,2021-07-15,Stock markets rallied after the company report...
1,2020-07-12,Apple stock is expected to be stable
2,2000-08-15,Amazon stock will highly fall


In [7]:
# Perform sentiment analysis on my dataframe
reports.reports_sentiment_analysis()

Device set to use cpu
Detecting sentiments: 100%|██████████| 1/1 [00:00<00:00,  1.37it/s]

Sentiment Analysis Report:
	- Positive: 1 tweets (Avg. score: 0.93)
	- Neutral: 1 tweets (Avg. score: 0.94)
	- Negative: 1 tweets (Avg. score: 0.75)


In [25]:
# Quick and More obvious example of how sentiment
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="ProsusAI/finbert", device=0)

text = "Apple stock is expected to be stable"
output = classifier(text)
print(output)


Device set to use cpu


[{'label': 'neutral', 'score': 0.9414412975311279}]
